In [19]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)
from sklearn.model_selection import GridSearchCV

In [20]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_fill_t.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_fill_t.csv')
testData = pd.read_csv(testPath)
submPath = os.path.join(homePath, 'submission.csv')
submData = pd.read_csv(submPath)

In [21]:
trainData['acc_now_delinq'] = list(map(int, trainData['acc_now_delinq']))

In [22]:
trainLabel = trainData['acc_now_delinq']
del trainData['acc_now_delinq']

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.1,
                     n_estimators=1000,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     reg_alpha=0.001,
                     reg_lambda=0.001,
                     scale_pos_weight=1)

import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
                                 max_features='sqrt', subsample=0.7, random_state=10)

vot = VotingClassifier(estimators=[('xgb', xgb), ('lgb', lgb_clf)], voting='soft')

In [ ]:
vot.fit(trainData, trainLabel)

In [ ]:
vot_pred = vot.predict(testData)

In [ ]:
vot_pred = pd.DataFrame(vot_pred)

In [ ]:
testData['acc_now_delinq'] = vot_pred

In [ ]:
testData['acc_now_delinq'].value_counts()

In [17]:
submData['acc_now_delinq'] = vot_pred

In [18]:
submData.to_csv(os.path.join(homePath, '起名字是最难的.csv'), index=False)